# How to save and load R objects from the workspace bucket

R users like to save intermediate work in R's native format for rapid loading.

See also [Notebooks 101 - How not to lose data output files or collaborator edits](https://broadinstitute.zendesk.com/hc/en-us/articles/360027300571-Notebooks-101-How-not-to-lose-data-output-files-or-collaborator-edits).

## Setup

In [1]:
library(lubridate)
library(tidyverse)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ dplyr   1.0.6
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::as.difftime() masks base::as.difftime()
✖ lubridate::date()        masks base::date()
✖ dplyr::filter()          masks stats::filter()
✖ lubridate::intersect()   masks base::intersect()
✖ dplyr::lag()             masks stats::lag()
✖ lubridate::setdiff()     masks base::setdiff()
✖ lubridate::union()       masks base::union()



Get the Cloud Storage bucket associated with this workspace.

In [2]:
(WORKSPACE_BUCKET <- Sys.getenv('WORKSPACE_BUCKET'))

[1] "gs://fc-ce0a87d7-b388-489d-9169-79ad4aa2e58d"

In [3]:
TIMESTAMP <- strftime(now(), '%Y%m%d-%H%M%S')
(RDA_FILE <- str_glue('thousand_genomes_{TIMESTAMP}.rda'))

thousand_genomes_20210519-234220.rda

## Read some data from Cloud Storage.
Let’s retrieve the sample information for [1000 Genomes](http://www.internationalgenome.org/data "1000 Genomes").

This approach uses `gsutil cat` to transfer the contents of the CSV file since we want to load the whole thing. 

If you instead want to load a subset of columns or a subset of rows, instead retrieve the data from BigQuery table [bigquery-public-data.human_genome_variants.1000_genomes_sample_info](https://bigquery.cloud.google.com/table/bigquery-public-data:human_genome_variants.1000_genomes_sample_info).

In [4]:
sample_info <- read_csv(pipe('gsutil cat gs://genomics-public-data/1000-genomes/other/sample_info/sample_info.csv'),
               guess_max = 5000)


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_character(),
  In_Low_Coverage_Pilot = col_double(),
  In_High_Coverage_Pilot = col_double(),
  In_Exon_Targetted_Pilot = col_double(),
  Has_Sequence_in_Phase1 = col_double(),
  In_Phase1_Integrated_Variant_Set = col_double(),
  Has_Phase1_chrY_SNPS = col_double(),
  Has_phase1_chrY_Deletions = col_double(),
  Has_phase1_chrMT_SNPs = col_double(),
  Total_LC_Sequence = col_double(),
  LC_Non_Duplicated_Aligned_Coverage = col_double(),
  Total_Exome_Sequence = col_double(),
  X_Targets_Covered_to_20x_or_greater = col_double(),
  VerifyBam_E_Omni_Free = col_double(),
  VerifyBam_E_Affy_Free = col_double(),
  VerifyBam_E_Omni_Chip = col_double(),
  VerifyBam_E_Affy_Chip = col_double(),
  VerifyBam_LC_Omni_Free = col_double(),
  VerifyBam_LC_Affy_Free = col_double(),
  VerifyBam_LC_Omni_Chip = col_double(),
  VerifyBam_LC_Affy_Chip = col_double()
  # ... with 11 more columns
)
ℹ Use `s

## Save the object(s) to a local file.

In [5]:
save(sample_info, file = RDA_FILE)

## Transfer the file to the workspace bucket

In [6]:
system(str_glue('gsutil cp {RDA_FILE} {WORKSPACE_BUCKET}/r-objects/ 2>&1'), intern = TRUE)

[1] "Copying file://thousand_genomes_20210519-234220.rda [Content-Type=application/octet-stream]..."                                                                      
[2] "/ [0 files][    0.0 B/173.0 KiB]                                                \r/ [1 files][173.0 KiB/173.0 KiB]                                                \r"
[3] "Operation completed over 1 objects/173.0 KiB.                                    "

## Now, load that object from the native format file in Cloud Storage

In [7]:
# The object exists in memory.
head(sample_info)

Sample,Family_ID,Population,Population_Description,Gender,Relationship,Unexpected_Parent_Child,Non_Paternity,Siblings,Grandparents,⋯,In_Final_Phase_Variant_Calling,Has_Omni_Genotypes,Has_Axiom_Genotypes,Has_Affy_6_0_Genotypes,Has_Exome_LOF_Genotypes,EBV_Coverage,DNA_Source_from_Coriell,Has_Sequence_from_Blood_in_Index,Super_Population,Super_Population_Description
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
HG00096,HG00096,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,20.31,NA,NA,EUR,European
HG00097,HG00097,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,169.49,NA,NA,EUR,European
HG00098,HG00098,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,NA,1,NA,NA,1,NA,NA,NA,EUR,European
HG00099,HG00099,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,23.04,NA,NA,EUR,European
HG00100,HG00100,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,116.22,NA,NA,EUR,European
HG00101,HG00101,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,82.00,NA,NA,EUR,European


In [8]:
# Go ahead and delete it.
rm(sample_info)

In [9]:
# Okay, its gone.
head(sample_info)

ERROR: Error in head(sample_info): object 'sample_info' not found


In [10]:
load(pipe(str_glue('gsutil cat {WORKSPACE_BUCKET}/r-objects/{RDA_FILE}')))

In [11]:
# The object exists in memory again!
head(sample_info)

Sample,Family_ID,Population,Population_Description,Gender,Relationship,Unexpected_Parent_Child,Non_Paternity,Siblings,Grandparents,⋯,In_Final_Phase_Variant_Calling,Has_Omni_Genotypes,Has_Axiom_Genotypes,Has_Affy_6_0_Genotypes,Has_Exome_LOF_Genotypes,EBV_Coverage,DNA_Source_from_Coriell,Has_Sequence_from_Blood_in_Index,Super_Population,Super_Population_Description
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
HG00096,HG00096,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,20.31,NA,NA,EUR,European
HG00097,HG00097,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,169.49,NA,NA,EUR,European
HG00098,HG00098,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,NA,1,NA,NA,1,NA,NA,NA,EUR,European
HG00099,HG00099,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,23.04,NA,NA,EUR,European
HG00100,HG00100,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,116.22,NA,NA,EUR,European
HG00101,HG00101,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,82.00,NA,NA,EUR,European


# Provenance

In [12]:
devtools::session_info()

─ Session info ───────────────────────────────────────────────────────────────
 setting  value                       
 version  R version 4.0.5 (2021-03-31)
 os       Ubuntu 18.04.5 LTS          
 system   x86_64, linux-gnu           
 ui       X11                         
 language (EN)                        
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       Etc/UTC                     
 date     2021-05-19                  

─ Packages ───────────────────────────────────────────────────────────────────
 package     * version date       lib source        
 assertthat    0.2.1   2019-03-21 [2] CRAN (R 4.0.5)
 backports     1.2.1   2020-12-09 [2] CRAN (R 4.0.5)
 base64enc     0.1-3   2015-07-28 [2] CRAN (R 4.0.5)
 broom         0.7.6   2021-04-05 [2] CRAN (R 4.0.5)
 cachem        1.0.4   2021-02-13 [2] CRAN (R 4.0.5)
 callr         3.7.0   2021-04-20 [2] CRAN (R 4.0.5)
 cellranger    1.1.0   2016-07-27 [2] CRAN (R 4.0.5)
 cli           2.5.0   2021

Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.